# What is the perfect anatomy of a powerlifter?

[Data source](https://openpowerlifting.gitlab.io/opl-csv/bulk-csv.html)

In [ ]:
import pandas as pd

df = pd.read_csv('data/openpowerlifting-2025-05-03-c77c7449.csv')

print(df.head())

print(df.info())

print(df.describe(include='all'))

print(df.isnull().sum())

In [ ]:
# Top lifters by total weight lifted
top_lifters = df[df['TotalKg'].notnull()].sort_values(by='TotalKg', ascending=False).head(10)
print(top_lifters[['Name', 'Sex', 'TotalKg', 'WeightClassKg', 'Federation']])

# Average lifts by gender
avg_by_sex = df.groupby('Sex')[['Best3SquatKg', 'Best3BenchKg', 'Best3DeadliftKg', 'TotalKg']].mean()
print(avg_by_sex)

# Most common federations
print(df['Federation'].value_counts().head(10))